Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

### Intel NLP-Architect ABSA on AzureML 

This notebook contains an end-to-end walkthrough of using Azure Machine Learning Service to train, finetune and test [Aspect Based Sentiment Analysis Models using Intel's NLP Architect](http://nlp_architect.nervanasys.com/absa.html)

### Prerequisites

* Understand the architecture and terms introduced by Azure Machine Learning (AML)
* Have working Jupyter Notebook Environment. You can:
    - Install Python environment locally, as described below in **Local Installation**
    - Use [Azure Notebooks](https://docs.microsoft.com/ru-ru/azure/notebooks/azure-notebooks-overview/?wt.mc_id=absa-notebook-abornst). In this case you should upload the `absa.ipynb` file to a new Azure Notebooks project, or just clone the [GitHub Repo](https://github.com/microsoft/ignite-learning-paths/tree/master/aiml/aiml40).
* Azure Machine Learning Workspace in your Azure Subscription

#### Local Installation

Install the Python SDK: make sure to install notebook, and contrib:

```shell
conda create -n azureml -y Python=3.6
source activate azureml
pip install --upgrade azureml-sdk[notebooks,contrib] 
conda install ipywidgets
jupyter nbextension install --py --user azureml.widgets
jupyter nbextension enable azureml.widgets --user --py
```

You will need to restart jupyter after this Detailed instructions are [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python/?WT.mc_id=absa-notebook-abornst)

If you need a free trial account to get started you can get one [here](https://azure.microsoft.com/en-us/offers/ms-azr-0044p/?WT.mc_id=absa-notebook-abornst)

#### Creating Azure ML Workspace

Azure ML Workspace can be created by using one of the following ways:
* Manually through [Azure Portal](http://portal.azure.com/?WT.mc_id=absa-notebook-abornst) - [here is the complete walkthrough](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace/?wt.mc_id=absa-notebook-abornst)
* Using [Azure CLI](https://docs.microsoft.com/ru-ru/cli/azure/?view=azure-cli-latest&wt.mc_id=absa-notebook-abornst), using the following commands:

```shell
az extension add -n azure-cli-ml
az group create -n absa -l westus2
az ml workspace create -w absa_space -g absa
```

## Initialize workspace

To access an Azure ML Workspace, you will need to import the AML library and the following information:
* A name for your workspace (in our example - `absa_space`)
* Your subscription id (can be obtained by running `az account list`)
* The resource group name (in our case `absa`)

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace/?WT.mc_id=absa-notebook-abornst) object from the existing workspace you created in the Prerequisites step or create a new one. 

In [20]:
from azureml.core import Workspace

#subscription_id = ''
#resource_group  = 'absa'
#workspace_name  = 'absa_space'
#ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#ws.write_config()

try:
    ws = Workspace.from_config()
    print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')
    print('Library configuration succeeded')
except:
    print('Workspace not found')

ariaiwork	eastus	ai_work	eastus
Library configuration succeeded


## Compute

There are two computer option run once(preview) and persistent compute for this demo we will use persistent compute to learn more about run once compute check out the [docs](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute?WT.mc_id=absa-notebook-abornst).

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cluster_name = "absa-cluster"

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           vm_priority='lowpriority',
                                                           min_nodes=1,
                                                           max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

## Upload Data

The dataset we are using comes from the [womens ecommerce clothing reviews dataset](https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews/) and is in the open domain, this can be replaced with any csv file with rows of text as the absa model is unsupervised. 

The documentation for uploading data can be found [here](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.azure_storage_datastore.azureblobdatastore/?WT.mc_id=absa-notebook-abornst) for now we will us the ds.upload command. 

In [ ]:
!wget -O 'dataset/glove.840B.300d.zip' 'http://nlp.stanford.edu/data/glove.840B.300d.zip'

In [ ]:
import os                            
lib_root = os.path.dirname(os.path.abspath("__file__"))
ds = ws.get_default_datastore()
ds.upload('./dataset', target_path='clothing_data', overwrite=True, show_progress=True)

Now the the glove file is uploaded to our datastore we can remove it from our local directory.

In [31]:
!rm 'dataset/glove.840B.300d.zip'

## Train File

In [ ]:
%%writefile train.py
import argparse
import json
import os 
from pathlib import Path
from nltk import flatten
from azureml.core import Run
from sklearn.metrics import f1_score
from azureml.core.model import Model

# Load NLP Architect
from nlp_architect.models.absa.train.train import TrainSentiment
from nlp_architect.models.absa.inference.inference import SentimentInference

# Inputs
parser = argparse.ArgumentParser(description='ABSA Train')
parser.add_argument('--data_folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--asp_thresh', type=int, default=3)
parser.add_argument('--op_thresh', type=int, default=2)
parser.add_argument('--max_iter', type=int, default=3)

args = parser.parse_args()

# Download ABSA dependencies including spacy parser and glove embeddings 
from spacy.cli.download import download as spacy_download
from nlp_architect.utils.io import uncompress_file
from nlp_architect.models.absa import TRAIN_OUT

spacy_download('en')
GLOVE_ZIP = os.path.join(args.data_folder, 
                                 'clothing_data/glove.840B.300d.zip')
EMBEDDING_PATH = TRAIN_OUT / 'word_emb_unzipped' / 'glove.840B.300d.txt'


uncompress_file(GLOVE_ZIP, Path(EMBEDDING_PATH).parent)

clothing_train = os.path.join(args.data_folder, 
                                 'clothing_data/clothing_absa_train_small.csv')

os.makedirs('outputs', exist_ok=True)

train = TrainSentiment(asp_thresh=args.asp_thresh,
                       op_thresh=args.op_thresh, 
                       max_iter=args.max_iter)

opinion_lex, aspect_lex = train.run(data=clothing_train,
                                    out_dir = './outputs')

# Evaluation 
# Although ABSA is an unsupervised method it can be metriced with a small sample of labeled data
def doc2IO(doc):
    """
    Converts ABSA doc to IO span format for evaluation
    """
    index = 0
    aspect_indexes = []
    doc_json = json.loads(doc.json())
    tokens = doc_json["_doc_text"].split()
    io = [[t,'O'] for t in tokens]
    for t_index, token in enumerate(tokens):
        for s in doc_json["_sentences"]:
            for ev in s["_events"]:
                for e in ev:
                    if e["_type"] == "ASPECT":
                        if e["_start"] == index and all(aspect[0] != t_index for aspect in aspect_indexes):
                            io[t_index][1] = "{}-{}".format(e["_text"], e["_polarity"])
        index += len(token) + 1
    
    return io

inference = SentimentInference('./outputs/train_out/generated_aspect_lex.csv', 
                               './outputs/train_out/generated_opinion_lex_reranked.csv')

clothing_val = os.path.join(args.data_folder, 
                                 'clothing_data/clothing-absa-validation.json')

with open(clothing_val) as json_file:
    val = json.load(json_file)

predictions = []
for doc in val["data"]:
    doc_raw = " ".join([token[0] for token in doc])
    sentiment_doc = inference.run(doc=doc_raw)
    predictions.append(doc2IO(sentiment_doc))
    
y_pred = flatten(predictions)[1::2]
y_true = flatten(val['data'])[1::2]

from sklearn.metrics import f1_score

# Log metrics
run = Run.get_context()
run.log('Aspect Lexicon Size', len(aspect_lex))
run.log('Opinion Lexicon Size', len(opinion_lex))
run.log('f1_weighted', float(f1_score(y_true, y_pred, average='weighted')))

## Create An Expierment

Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment/?WT.mc_id=absa-notebook-abornst) to track all the runs in your workspace for this distributed PyTorch tutorial. 

In [23]:
from azureml.core import Experiment
experiment_name = 'absa'
exp = Experiment(workspace=ws, name=experiment_name)

In [ ]:
from azureml.train.estimator import Estimator

script_params = {
    '--data_folder': ds,
}

nlp_est = Estimator(source_directory='.',
                   script_params=script_params,
                   compute_target=cluster,
                   environment_variables = {'NLP_ARCHITECT_BE':'CPU'},
                   entry_script='train.py',
                   pip_packages=['git+https://github.com/NervanaSystems/nlp-architect.git@absa',
                                 'spacy==2.1.8']
)

To create a run we just submit our expierment as follows.

In [ ]:
run = exp.submit(nlp_est)

Note: If you accidently run the following cell more than once you can cancel a run with the run.cancel() command.

In [ ]:
# run.cancel()

You can load any previous run using its run id

In [ ]:
run.id

In [24]:
run = [r for r in exp.get_runs() if r.id == 'absa_1570979110_e5021352'][0]

Let's visualize our run:

In [25]:
from azureml.widgets import RunDetails

RunDetails(run).show()

A Jupyter Widget

## Fine-Tuning NLP Archictect  with AzureML HyperDrive
Although ABSA is an unsupervised method it's hyper parameters such as the aspect and opinion word thresholds can be fined tuned if provided with a small sample of labeled data

In [ ]:
from azureml.train.hyperdrive import *
import math

param_sampling = RandomParameterSampling({
         '--asp_thresh': choice(range(2,5)),
         '--op_thresh': choice(range(2,5)), 
         '--max_iter': choice(range(2,5))
    })

### Early Termination Policy
First we will define an early terminination policy. [Median stopping](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.medianstoppingpolicy?WT.mc_id=absa-notebook-abornst) is an early termination policy based on running averages of primary metrics reported by the runs. This policy computes running averages across all training runs and terminates runs whose performance is worse than the median of the running averages. 

This policy takes the following configuration parameters:

- evaluation_interval: the frequency for applying the policy (optional parameter).
- delay_evaluation: delays the first policy evaluation for a specified number of intervals (optional parameter).


In [ ]:
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=0)

Refer [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters#specify-early-termination-policy?WT.mc_id=absa-notebook-abornst) for more information on the Median stopping policy and other policies available.

Now that we've defined our early termination policy we can define our Hyper Drive configuration to maximize our Model's weighted F1 score. Hyper Drive can optimize any metric can be optimized as long as it's logged by the training script. 


In [ ]:
hd_config = HyperDriveConfig(estimator=nlp_est,
                            hyperparameter_sampling=param_sampling,
                            policy=early_termination_policy,
                            primary_metric_name='f1_weighted',
                            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                            max_total_runs=16,
                            max_concurrent_runs=4)

Finally, lauch the hyperparameter tuning job.

In [ ]:
experiment = Experiment(workspace=ws, name='absa_hyperdrive')

In [ ]:
hyperdrive_run = experiment.submit(hd_config)

In [ ]:
hyperdrive_run.id

In [ ]:
hyperdrive_run = [r for r in experiment.get_runs() if r.id == 'absa_hyperdrive_1571092544235933'][0]

### Monitor HyperDrive runs
We can monitor the progress of the runs with the following Jupyter widget. 

In [ ]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.cancel()

### Find and register the best model
Once all the runs complete, we can find the run that produced the model with the highest evaluation (METRIC TBD).

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)
print('Best Run is:\n  F1: {0:.5f}'.format(
        best_run_metrics['f1_weighted']
     ))

## Register Model Outputs

In [ ]:
aspect_lex = run.register_model(model_name='c_aspect_lex', model_path='outputs/train_out/generated_aspect_lex.csv')
opinion_lex = run.register_model(model_name='c_opinion_lex', model_path='outputs/train_out/generated_opinion_lex_reranked.csv')

## Test Locally

### Install Local PIP Dependencies

In [ ]:
!pip install git+https://github.com/NervanaSystems/nlp-architect.git@absa

In [ ]:
!pip install spacy==2.0.18

### Load Model From AzureML

In [26]:
from azureml.core.model import Model
from nlp_architect.models.absa.inference.inference import SentimentInference
c_aspect_lex = Model._get_model_path_remote('c_aspect_lex', 1, ws)
c_opinion_lex = Model._get_model_path_remote('c_opinion_lex', 1, ws)   
inference = SentimentInference(c_aspect_lex, c_opinion_lex)


Using pre-trained BIST model.


### Run Model On Sample Data 

In [27]:
docs = ["Loved the sweater but hated the pants",
       "Really great outfit, but the shirt is the wrong size",
       "I absolutely love this jacket! i wear it almost everyday. works as a cardigan or a jacket. my favorite retailer purchase so far"]

sentiment_docs = []

for doc_raw in docs:
    sentiment_doc = inference.run(doc=doc_raw)
    sentiment_docs.append(sentiment_doc)

### Visualize Model Results

In [28]:
import spacy
from spacy import displacy
from nlp_architect.models.absa.inference.data_types import TermType
ents = []
for doc in sentiment_docs:    
    if doc:
        doc_viz = {'text':doc._doc_text, 'ents':[]}
        for s in doc._sentences:
            for ev in s._events:
                for e in ev:
                    if e._type == TermType.ASPECT:
                        ent = {'start': e._start, 'end': e._start + e._len,
                               'label':str(e._polarity.value), 
                               'text':str(e._text)}
                        if all(kown_e['start'] != ent['start'] for kown_e in ents):
                            ents.append(ent)
                            doc_viz['ents'].append(ent)
        doc_viz['ents'].sort(key=lambda m: m["start"])
        displacy.render(doc_viz, style="ent", options={'colors':{'POS':'#7CFC00', 'NEG':'#FF0000'}}, manual=True)

# Package Model For Deployment

## Create scoring script
Create the scoring script, called score.py, used by the web service call to show how to use the model.

You must include two required functions into the scoring script:

The init() function, which typically loads the model into a global object. This function is run only once when the Docker container is started.

The run(input_data) function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.

In [ ]:
%%writefile score.py
from azureml.core.model import Model
from nlp_architect.models.absa.inference.inference import SentimentInference
from spacy.cli.download import download as spacy_download


def init():
    """
    Set up the ABSA model for Inference  
    """
    global SentInference
    spacy_download('en')
    aspect_lex = Model.get_model_path('c_aspect_lex')
    opinion_lex = Model.get_model_path('c_opinion_lex') 
    SentInference = SentimentInference(aspect_lex, opinion_lex)

def run(raw_data):
    """
    Evaluate the model and return JSON string
    """
    sentiment_doc = SentInference.run(doc=raw_data)
    return sentiment_doc.json()

## Create configuration files


### Create Enviorment File
create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs nlp-architect and the azureml-sdk. 

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

pip = ["azureml-defaults", "azureml-monitoring", 
       "git+https://github.com/NervanaSystems/nlp-architect.git@absa", 
       "spacy==2.0.18"]

myenv = CondaDependencies.create(pip_packages=pip)

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

### Create Environment Config
Create a Enviorment configuration file and specify the enviroment and enviormental variables required for the application

In [ ]:
from azureml.core import Environment
deploy_env = Environment.from_conda_specification('absa_env', "myenv.yml")
deploy_env.environment_variables={'NLP_ARCHITECT_BE': 'CPU'}

### Inference Config 
Create an inference configuration that recieves the deployment enviorment and the entry script

In [ ]:
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(environment=deploy_env,
                                   entry_script="score.py")

### Package Model and Pull 
Create an inference configuration that recieves the deployment enviorment and the entry script

In [ ]:
package = Model.package(ws, [aspect_lex, opinion_lex], inference_config)
package.wait_for_creation(show_output=True)


In [ ]:
package.pull()

## Next Steps

We now have gone through all the steps for production training of a custom open source model using the AzureML Service check out AIML50 to learn how to deploy and models and manage re-training pipelines